<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Anton_Shalin/Grounding_Dino_Labeling_for_YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Модуль разбора видео на кадры

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00


In [ ]:
!rm -r sample_data

In [ ]:
import cv2
import os
import shutil
from google.colab import drive
from pytube import YouTube

In [ ]:
step_ms = 250 # Интервал извлечения кадров в мсек

# Параметр source определяет откуда будем парсить: из YouTube или сохраненное на Google Disk

source = 'local' # Читаем файл из Google Drive, иначе из YouTube
#source = 'YouTube'

# Монтирование Google Drive
drive.mount('/content/drive')

capture_path = '/content/drive/MyDrive/UII/Capture/'
video_path = '/content/drive/MyDrive/UII/Video/'

#video_url = "https://www.youtube.com/watch?v=5wwH7Hll0xE&pp=ygUOWW9nYSB0aW1lbGFwc2U%3D"  # YouTube Yoga Timelapse
#video_url = "https://www.youtube.com/watch?v=0pKQ8E5cFIE"  # Подскальзывания
#video_url = "https://www.youtube.com/watch?v=GZcghScPe6Y" # Подборка разных падений
video_url = "https://www.youtube.com/watch?v=o-N3ImykVuI" # Падения разные

# Функция очистки папки от предыдущих результатов
def cleaner_folder(path):
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Не удалось удалить %s. Причина: %s' % (file_path, e))

# очистим папку Capture от предыдущих результатов
cleaner_folder(capture_path)

# Проверка существования пути для видео, если нет, то создаем
if not os.path.exists(video_path):
    os.makedirs(video_path)

# Проверка существования пути для кадров, если нет, то создаем
if not os.path.exists(capture_path):
    os.makedirs(capture_path)

# Функция для скачивания видео с YouTube
def download_youtube_video(url, path):
    yt = YouTube(url)
    # Выбор потока с наивысшим разрешением
    video_stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    video_stream.download(path)
    return os.path.join(path, video_stream.default_filename)

# Функция выделения кадров из видео
def extract_frames(video_file, step_ms):
    vidcap = cv2.VideoCapture(video_file)
    success, image = vidcap.read()
    count = 0

    print(f"Начало обработки видео: {video_file}")

    while success:
        # Сохранение кадра каждые 'step_ms' миллисекунд
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count * step_ms))
        success, image = vidcap.read()
        if success:
            # Формирование имени файла кадра
            frame_filename = f"{os.path.splitext(os.path.basename(video_file))[0]}_frame{count}.png"
            cv2.imwrite(os.path.join(capture_path, frame_filename), image)
            count += 1

    print(f"Обработка видео завершена: {video_file}")

if source == 'local':
    # Чтение и обработка каждого файла в папке
    for filename in os.listdir(video_path):
        if filename.endswith(".mp4"): # Проверка, что файл является видео
            extract_frames(os.path.join(video_path, filename), step_ms)
else:
    downloaded_video_path = download_youtube_video(video_url, video_path)
    extract_frames(downloaded_video_path, step_ms)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Начало обработки видео: /content/drive/MyDrive/UII/Video/Люди поскальзываются на неубранном льду.mp4
Обработка видео завершена: /content/drive/MyDrive/UII/Video/Люди поскальзываются на неубранном льду.mp4
Начало обработки видео: /content/drive/MyDrive/UII/Video/morning yoga time lapse.mp4
Обработка видео завершена: /content/drive/MyDrive/UII/Video/morning yoga time lapse.mp4


## Модуль детекции объектов по текстовому описанию на картинках. Используется Grounding Dino

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=49a176160caa827cf39a201028e4793d42ccee749046f399d9f1f43943c09e11
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import torch
import numpy as np
import random
import os
import shutil
import wget
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw
%matplotlib inline
import math
import warnings
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 421 (delta 144), reused 126 (delta 126), pack-reused 232
Receiving objects: 100% (421/421), 12.85 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (216/216), done.


In [ ]:
%cd {HOME}/GroundingDINO
!pip install -e .

/content/GroundingDINO
Obtaining file:///content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.0 MB/s eta 0:00:00
  Running setup.py develop for groundingdino


In [ ]:
# загрузка весов GroundingDINO

%cd {HOME}
os.makedirs(os.path.join(HOME, 'weights'), exist_ok=True)
%cd {HOME}/weights
#!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
url = 'https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth'
wget.download(url, os.getcwd())

/content
/content/weights


'/content/weights/groundingdino_swint_ogc.pth'

In [ ]:
# Загрузка модели GroundingDINO

%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate
import supervision as sv

/content/GroundingDINO


In [ ]:
# глобальные параметры и переменные

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))
BOX_TRESHOLD = 0.43
TEXT_TRESHOLD = 0.45

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
def transform_bbox_coords(bbox):
    '''
    Трансформация координат ббокса из фотмата х_центра, у_центра, ширина, высота в x_min, y_min, x_max, y_max.
    '''
    x_cent, y_cent, width, height = bbox
    x_min = int(x_cent - width/2)
    y_min = int(y_cent - height/2)
    x_max = int(x_cent + width/2)
    y_max = int(y_cent + height/2)
    return [x_min, y_min, x_max, y_max]

In [ ]:
def grounding_dino_inference(image_path,
                             text_prompt,
                             bbox_lines_whidth=3,
                             different_bbox_colors_on_image=False,
                             return_image_only=False):

    '''
    Функция инференса GroundingDINO по поданному промпту с отрисовкой результата
    '''

    image_source, image = load_image(image_path)

    boxes, logits, phrases = predict(model=model,
                                     image=image,
                                     caption=text_prompt,
                                     box_threshold=BOX_TRESHOLD,
                                     text_threshold=TEXT_TRESHOLD)

    # Проверяем, обнаружен ли объект
    object_detected = len(boxes) > 0

    if not return_image_only:
        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
        sv.plot_image(annotated_frame, (12, 12))
    else:
        imagePIL = Image.fromarray(image_source)
        bboxes = (boxes*torch.Tensor(imagePIL.size).tile((boxes.size()[0], int(boxes.size()[1]/2)))).to(dtype=torch.int16).tolist()
        color = tuple(np.random.randint((255, 255, 255)))
        draw = ImageDraw.Draw(imagePIL)
        for bbox in bboxes:
            bbox = transform_bbox_coords(bbox)
            if different_bbox_colors_on_image:
                color = tuple(np.random.randint((255, 255, 255)))
            draw.rectangle(bbox, outline=color, width=bbox_lines_whidth)
        return imagePIL, object_detected

In [ ]:
# Создание экземпляра модели
model = load_model(CONFIG_PATH, WEIGHTS_PATH)

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
capture_path = '/content/drive/MyDrive/UII/Capture/'
annotated_path = '/content/drive/MyDrive/UII/Capture/Annotated/'

# Проверка и создание пути для аннотированных изображений
if not os.path.exists(annotated_path):
    os.makedirs(annotated_path)

total_files = len(os.listdir(capture_path))
processed = 0

# Функция очистки папку от предыдущих результатов
def cleaner_folder(path):
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Не удалось удалить %s. Причина: %s' % (file_path, e))

cleaner_folder(annotated_path)

for filename in os.listdir(capture_path):
    if filename.endswith(".png"):  # Предположим, что кадры в формате PNG
        image_path = os.path.join(capture_path, filename)
        print(f"Обрабатывается файл {filename} ({processed + 1} из {total_files})")

        result_image, object_detected = grounding_dino_inference(image_path, 'fall', return_image_only=True)

        # Проверка наличия объекта
        if object_detected:
            annotated_image_path = os.path.join(annotated_path, filename)
            # Сохранение аннотированного изображения
            result_image.save(annotated_image_path)
            print(f"Объект обнаружен и сохранен в {annotated_image_path}")
        else:
            print(f"Объект не обнаружен на {filename}")

        processed += 1


Обрабатывается файл Люди поскальзываются на неубранном льду_frame0.png (1 из 1453)
Объект не обнаружен на Люди поскальзываются на неубранном льду_frame0.png
Обрабатывается файл Люди поскальзываются на неубранном льду_frame1.png (2 из 1453)
Объект не обнаружен на Люди поскальзываются на неубранном льду_frame1.png
Обрабатывается файл Люди поскальзываются на неубранном льду_frame2.png (3 из 1453)
Объект не обнаружен на Люди поскальзываются на неубранном льду_frame2.png
Обрабатывается файл Люди поскальзываются на неубранном льду_frame3.png (4 из 1453)
Объект не обнаружен на Люди поскальзываются на неубранном льду_frame3.png
Обрабатывается файл Люди поскальзываются на неубранном льду_frame4.png (5 из 1453)
Объект не обнаружен на Люди поскальзываются на неубранном льду_frame4.png
Обрабатывается файл Люди поскальзываются на неубранном льду_frame5.png (6 из 1453)
Объект обнаружен и сохранен в /content/drive/MyDrive/UII/Capture/Annotated/Люди поскальзываются на неубранном льду_frame5.png
Обраба